<a href="https://colab.research.google.com/github/projects-in-programming-f24/01_Data_Feature/blob/main/PPDS_Activity_01_Data_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Features: Portfolio News

In this exercise, we will build a simple data feature prototype that helps users find news articles related to a specific stock ticker.

The process of building this should help you think about interesting ways you might combine data and APIs in interesting ways for your assignment this week.

We will use two APIs:
1. An API to convert a stock ticker to the company name.
2. The NYTimes Article Search API to find news articles related to the company.

## Steps:
1. User enters a stock ticker symbol.
2. We use an API to convert the ticker to the company name.
3. We call the NYTimes API with the company name to retrieve related articles.

# Setting Up the APIs

First, we need to set up the API keys for the services we will use:
1. A service to convert stock tickers to company names.
2. The NYTimes Article Search API.

STEP 1: Sign up for the NYTimes API: https://developer.nytimes.com/
- then follow the steps here: https://developer.nytimes.com/get-started
- add your APPs `API KEY` to the `SECRETS` tab here in Google Colab

STEP 2: Sign up for an API Key at Alpha Vantage: https://www.alphavantage.co/
- add it to your `SECRETS` tab in collab too.

Make sure you have your API keys stored in the `SECRETS` tab on the left (key icon) and named like the variables provided below. The first code cell has you prove that they are present :)

In [ ]:
# Set up the API keys
from google.colab import userdata

# Retrieve the NYTimes API key
NYTIMES_API_KEY = userdata.get('NYTIMES_API_KEY')

# Set up the API key for the stock ticker to company name service
ALPHA_VANTAGE_KEY = userdata.get('ALPHA_VANTAGE_KEY')

# Print confirmation messages, do not print API keys (bad practice!)
if NYTIMES_API_KEY:
    print("NYTimes API key retrieved successfully!")
else:
    print("Failed to retrieve NYTimes API key. Please check your setup.")

if ALPHA_VANTAGE_KEY:
    print("Stock API key is set!")
else:
    print("Failed to set Stock API key. Please check your setup.")

NYTimes API key retrieved successfully!
Stock API key is set!


# Convert Ticker to Company Name

We will use a simple call to the Alpha Vantage API to convert a stock ticker symbol (e.g., AAPL) into the corresponding company name (e.g., Apple Inc.).

They have an endpoint called `SYMBOL_SEARCH` that allows us to search for a ticker and returns the company name.



In [ ]:
# Let's just see what the data looks like
# API Documentation: https://www.alphavantage.co/documentation/

import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=tesco&apikey={ALPHA_VANTAGE_KEY}'
r = requests.get(url)
data = r.json()

print(data)

{'bestMatches': [{'1. symbol': 'TSCO.LON', '2. name': 'Tesco PLC', '3. type': 'Equity', '4. region': 'United Kingdom', '5. marketOpen': '08:00', '6. marketClose': '16:30', '7. timezone': 'UTC+01', '8. currency': 'GBX', '9. matchScore': '0.7273'}, {'1. symbol': 'TSCDF', '2. name': 'Tesco plc', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.7143'}, {'1. symbol': 'TSCDY', '2. name': 'Tesco plc', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.7143'}, {'1. symbol': 'TCO2.FRK', '2. name': 'TESCO PLC ADR/1 LS-05', '3. type': 'Equity', '4. region': 'Frankfurt', '5. marketOpen': '08:00', '6. marketClose': '20:00', '7. timezone': 'UTC+02', '8. currency': 'EUR', '9. matchScore': '0.5455'}, {'1. symbol': 'TCO0.FRK', '2. name': 'TESCO PLC LS-0633333', '3.

In [ ]:
# okay, so it looks like we get JSON back where the key `bestMatches` has an array of key:value pairs, and `2. name` is the name associated with the ticker.
import requests

# Function to convert ticker to company name using Alpha Vantage API
def get_company_name(ticker):
    url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={ticker}&apikey={ALPHA_VANTAGE_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'bestMatches' in data and data['bestMatches']:
            return data['bestMatches'][0]['2. name']
        else:
            print("Company name not found. Please check the ticker symbol and try again.")
            return None
    else:
        print("Failed to retrieve company name. Please check the ticker symbol and try again.")
        return None

# Test this and see if we can store a company name
ticker = 'AAPL'
company_name = get_company_name(ticker)
print(f'The company name for ticker {ticker} is {company_name}')


The company name for ticker AAPL is Apple Inc


# Search for Articles Using NYTimes API

Now that we have the company name, we will use the NYTimes Article Search API to find news articles related to this company.


In [ ]:
# Function to search for articles
def search_articles(company_name):
    url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={company_name}&api-key={NYTIMES_API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['response']['docs']
    else:
        print("Failed to retrieve articles. Please try again.")
        return []

# Test the function, we should get 10 articles back
articles = search_articles(company_name)
print(f'Found {len(articles)} articles related to {company_name}')

# Display 3 articles so we can test this
for article in articles[:3]:
    print(f"Title: {article['headline']['main']}")
    print(f"URL: {article['web_url']}\n")


Found 10 articles related to Apple Inc
Title: Berkshire’s Cash Stockpile Soars as It Cuts Its Stake in Apple
URL: https://www.nytimes.com/2024/08/03/business/berkshire-hathaway-buffett-earnings.html

Title: Apple Store Workers Get First U.S. Contract
URL: https://www.nytimes.com/2024/08/06/business/economy/apple-store-union-contract.html

Title: Sales of Macs and iPads Lead Apple’s Quarterly Revenue Up 5%
URL: https://www.nytimes.com/2024/08/01/technology/apple-quarterly-revenue-up-5.html



# Putting It All Together

We have now built the components to:
1. Convert a stock ticker to a company name.
2. Search for news articles related to the company.

Let's combine these steps into a single function that takes a ticker symbol as input and returns the related news articles.

Note: In a real data product, we would have fancy things like a UI, the ability to add and remove tickers, financial metrics, robust error handling, etc..., but for now though we can simply have the user enter a ticker and then display 10 articles


In [ ]:
# Combined function to get the ticker's company and the articles
def get_articles_for_ticker(ticker):
    company_name = get_company_name(ticker)
    if company_name:
        articles = search_articles(company_name)
        return articles
    else:
        return [] # return an empty array

# Function to display news articles
def display_articles(articles):
    if articles:
        print(f'Found {len(articles)} articles:')
        for article in articles[:10]:  # Display 10 articles
            print(f"Title: {article['headline']['main']}")
            print(f"URL: {article['web_url']}\n")
    else:
        print("No articles found.")

# User input for ticker symbol
ticker = input("Enter a stock ticker symbol: ")
articles = get_articles_for_ticker(ticker)
display_articles(articles)

Enter a stock ticker symbol: BBER
Company name not found. Please check the ticker symbol and try again.
No articles found.
